In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [ ]:
# load the dataset
df = pd.read_csv('../preprocessed_dataset/dataset_preprocessed.csv')

# map hate to 1 and non-hate to 0
df['label'] = df['label'].map({'hate': 1, 'not_hate': 0})

sentences = df['text'].values
labels = df['label'].values


# split the dataset 0.6, 0.2, 0.2
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2, random_state=42)
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_sentences, train_labels, test_size=0.2, random_state=42)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")

# Define label mapping
label_mapping = {
    0: "not_hate",
    1: "hate"
}

# Encode the input text
inputs = tokenizer("I kill arabs.", return_tensors="pt")

# Get the model outputs
outputs = model(**inputs)

# Extract predicted label and score
predicted_label = torch.argmax(outputs.logits, dim=1).item()
predicted_label_text = label_mapping[predicted_label]
predicted_score = torch.softmax(outputs.logits, dim=1)[0][predicted_label].item()

print("Softmax:", torch.softmax(outputs.logits, dim=1))
print("Predicted Label:", predicted_label_text)
print("Predicted Score:", predicted_score)